# S4 TDS analysis

In [8]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 

In [9]:
#reading vaclogger measurement file
vaclog=pd.read_csv("vaclog", sep="\t")


#adding an elapsed time column
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog.head()

,Live comments,Time,Elapsed time,100mbar inj,Barion_2,Barion_1,DUAL experiment,DUAL insulation,1mbar inj,helium,T-platinum,T-CERNOX,V_heater,DUAL inj line,I_emission,I_grid
0,NaN,2023-05-08 17:31:51,0.0,-0.021935,1.270000e-09,6.620000e-10,4.996000e-09,4.362000e-07,-0.000755,124.0,-1.347,3929.500,0.000009,0.000006,NaN,NaN
1,NaN,2023-05-08 17:32:00,9.0,-0.021294,1.270000e-09,6.620000e-10,4.996000e-09,4.362000e-07,-0.000791,124.0,-1.347,3934.999,0.000009,0.000006,NaN,NaN
2,NaN,2023-05-08 17:32:09,18.0,-0.021788,1.280000e-09,6.620000e-10,4.996000e-09,4.362000e-07,-0.000793,123.0,-1.459,3937.581,0.000009,0.000006,NaN,NaN
3,NaN,2023-05-08 17:32:17,26.0,-0.021419,1.270000e-09,6.620000e-10,4.996000e-09,4.362000e-07,-0.000792,123.0,-1.347,3905.370,0.000009,0.000006,NaN,NaN
4,NaN,2023-05-08 17:32:26,35.0,-0.025609,1.280000e-09,6.620000e-10,4.996000e-09,4.362000e-07,-0.000792,122.0,-1.347,3889.433,0.000008,0.000006,NaN,NaN


In [10]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [11]:
#print hv comments
print(pd.unique(vaclog["Live comments"]))
vaclog_comm = vaclog.loc[:,["Time","Live comments"]]
vaclog_comm.dropna(inplace=True)
print(vaclog_comm)
vaclog_comm.index

[nan 'Mass scan at cernox=3845 ohm']
                  Time                 Live comments
32 2023-05-08 17:36:34  Mass scan at cernox=3845 ohm


Int64Index([32], dtype='int64')

### Plotting

In [12]:
#define parameters for plotting
#annotation params
font = dict(size = "x-small", color ="green", style ="italic",rotation="vertical")


In [13]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(12,6))
plt.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.plot(vaclog["Time"],vaclog["DUAL experiment"],marker=".", markersize=4,label='Dual exp')
plt.plot(vaclog["Time"],vaclog["helium"],marker=".", markersize=4,label='Dual exp')

plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("S4 Total pressure evolution")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')  

#comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in vaclog_comm.index:
    plt.text(vaclog_comm["Time"][i],0.8,str(vaclog_comm["Live comments"][i]),fontdict=font, transform=trans)
    
plt.savefig(r'./graphs/TDS pressures.png')
plt.show()

In [14]:
## Plotting the temp evolution
plt.figure(figsize=(10,6))
plt.plot((vaclog["Time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')          
plt.xlabel('Timestamp')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("S4 TDS Temperature evolution")
plt.yscale('linear')
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
    
plt.savefig(r'./graphs/Temp.png')
plt.show()